# Data Preprocessing Tools

## Importing the libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [6]:
# Load Excel files
df = pd.read_csv("0_maintenance_raw_data/maintenance.csv", sep=";")

In [7]:
print(df.columns)

Index(['lifetime', 'broken', 'pressureInd', 'moistureInd', 'temperatureInd',
       'team', 'provider'],
      dtype='object')


In [8]:
print(df.head())

   lifetime  broken  pressureInd  moistureInd  temperatureInd   team  \
0        56       0    92.178854   104.230204       96.517159  TeamA   
1        81       1    72.075938   103.065701       87.271062  TeamC   
2        60       0    96.272254    77.801376      112.196170  TeamA   
3        86       1    94.406461   108.493608       72.025374  TeamC   
4        34       0    97.752899    99.413492      103.756271  TeamB   

    provider  
0  Provider4  
1  Provider4  
2  Provider1  
3  Provider2  
4  Provider1  


In [9]:
print(df.shape)

(1000, 7)


## Export Updated Excel

In [10]:
# Create the folder if it doesn't exist
os.makedirs("1_maintenance_survival_data", exist_ok=True)

# Save the Excel file inside the folder
df.to_csv("1_maintenance_survival_data/1_maintenance_survival_data.csv", index=False)